# Common code necessary to run

### Install packages

In [1]:
print()


In [2]:
import tensorflow as tf
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

2023-11-28 15:13:47.730522: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Using TensorFlow backend.


In [3]:
if tf.test.is_gpu_available():
    print("GPU is available")

if tf.test.gpu_device_name():
    print("GPU is being used for the current session")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


after install package remember to restart kernel

## Login to GEE

In [4]:
!pip install earthengine-api

In [41]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Nov 28 19:56:48 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     On  | 00000000:06:00.0 Off |                  N/A |
| 35%   48C    P8              17W / 250W |   6344MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [38]:
import ee
import os
# import folium
# import geemap
from pprint import pprint
# project = "wildfire-feature-importance"
project = "ee-my-char-test"
os.environ["GOOGLE_CLOUD_PROJECT"] = project

auth earth engine

In [40]:
ee.Authenticate()
ee.Initialize(
    project=project,
    opt_url="https://earthengine-highvolume.googleapis.com")


Successfully saved authorization token.


In [42]:
import tensorflow as tf
tf.io.gfile.exists("gs://tfds-data/dataset_info/mnist/3.0.1")

AbortedError: All 10 retry attempts failed. The last failure: Error executing an HTTP request: libcurl code 60 meaning 'SSL peer certificate or SSH remote key was not OK', error details: SSL certificate problem: unable to get local issuer certificate
	 when reading metadata of gs://tfds-data/dataset_info/mnist/3.0.1

## Define visualization palette

In [7]:
fire_vis = ['bbe029', '0a9501', '074b03']

landcover_vis = {
    "Evergreen Needleleaf Forests": "05450a",
    "Evergreen Broadleaf Forests": "086a10",
    "Deciduous Needleleaf Forests": "54a708",
    "Deciduous Broadleaf Forests": "78d203",
    "Mixed Forests": "009900",
    "Closed Shrublands": "c6b044",
    "Open Shrublands": "dcd159",
    "Woody Savannas": "dade48",
    "Savannas": "fbff13",
    "Grasslands": "b6ff05",
    "Permanent Wetlands": "27ff87",
    "Croplands": "c24f44",
    "Urban and Built-up Lands": "a5a5a5",
    "Cropland/Natural Vegetation Mosaics": "ff6d4c",\
    "Permanent Snow and Ice": "69fff8",
    "Barren": "f9ffa4",
    "Water Bodies": "1c0dff",
}

temp_vis = [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ]

rain_vis = ['d8d8d8', '4addff', '5affa3', 'f2ff89', 'ff725c']


ndvi_vis = [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ]


# Global ALOS Landforms
# topo_vis = [
#     'red', 'red', 'blue', 'blue', 'blue', 'blue',
#     'blue', 'blue', 'blue', 'blue', 'blue', 'blue',
#     'blue', 'blue', 'green', 'green', 'green', 'green',
#     'yellow', 'yellow', 'orange', 'red', 'red', 'red',
#     'red', 'red', 'red', 'red', 'red'
#   ]

cdem_vis = ['0905ff', 'ffefc4', 'ffffff']

windspeed_vis = ['ff00ff', '00ffff', 'ffff00']

soil_moisture_info_vis = ['ff5733', '1eae98', 'a693bf']

evapotranspiration_vis = ['0099cc', 'ffcc00', '9900cc']

## Define feature information

In [8]:
# FIRMS: Fire Information for Resource Management System
# 2000-2023 verified
# https://developers.google.com/earth-engine/datasets/catalog/FIRMS
fire_map_info = {
    'name':'FIRMS', 
    'band':'T21', 
    'min':325, 
    'max':400, 
    'vis':fire_vis, 
    'unmask':0,
    'is_label':-1,
}
fire_class_band = 'is_fire'


feature_list = []
# Terra Land Surface Temperature and Emissivity Daily Global 1km
# 2000-2023 verified
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD11A1
temp_map_info = {
    'name':'MODIS/061/MOD11A1', 
    'band':'LST_Day_1km', 
    'min':13000.0, 
    'max':16500.0, 
    'vis':temp_vis}
feature_list.append(temp_map_info)


# TerraClimate: Monthly Climate and Climatic Water Balance for Global Terrestrial Surfaces, 
# Precipitation accumulation 
# -2022.12 verified
# https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
rain_map_info = {
    'name':'IDAHO_EPSCOR/TERRACLIMATE', 
    'band':'pr', 
    'min':0, 
    'max':2000, 
    'vis':rain_vis}
feature_list.append(rain_map_info)


# MCD12Q1.061 MODIS Land Cover Type Yearly Global 500m, 
# 	Land Cover Type 1: Annual International Geosphere-Biosphere Programme (IGBP) classification
# -2022.1 verified
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD12Q1
landcover_map_info = {
    'name':'MODIS/061/MCD12Q1', 
    'band':'LC_Type1', 
    'min':1.0, 
    'max':17.0, 
    'vis':landcover_vis,
    'annual':True}
feature_list.append(landcover_map_info)


# MODIS Combined 16-Day NDVI
# 2000-2022.1 verified
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_MCD43A4_006_NDVI
ndvi_map_info = {
    'name':'MODIS/MCD43A4_006_NDVI', 
    'band':'NDVI', 
    'min':0, 
    'max':1, 
    'vis':ndvi_vis}
feature_list.append(ndvi_map_info)

# The ALOS Landform dataset provides landform classes created by combining the Continuous Heat-Insolation Load Index (ALOS CHILI) and the multi-scale Topographic Position Index (ALOS mTPI) datasets. It is based on the 30m "AVE" band of JAXA's ALOS DEM (available in EE as JAXA/ALOS/AW3D30_V1_1).
# 2006-01-24T00:00:00Z–2011-05-13T00:00:00
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD11A1
# topo_map_info = {
#     'name': 'CSP/ERGo/1_0/Global/ALOS_landforms',
#     'band': 'constant',
#     'min': 11, 
#     'max': 42, 
#     'vis': 'topo_vis',
#     'unmask': -1,
#     'topo': -1,
# }
# feature_list.append(topo_map_info)


# The Canadian Digital Elevation Model (CDEM) is part of Natural Resources Canada's (NRCan) altimetry system and stems from the existing Canadian Digital Elevation Data (CDED). In these data, elevations can be either ground or reflective surface elevations.
# 1945-01-01T00:00:00Z–2011-01-01T00:00:00 verified
# https://developers.google.com/earth-engine/datasets/catalog/NRCan_CDEM#description
cdem_map_info = {
    'name': 'NRCan/CDEM',
    'band': 'elevation',
    'min': -226, 
    'max': 5944, 
    'vis': 'cdem_vis',
    'no_time': -1,
}
feature_list.append(cdem_map_info)


# TerraClimate: Monthly Climate and Climatic Water Balance for Global Terrestrial Surfaces
# -2022.12 verified
# https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
windspeed_map_info = {
    'name':'IDAHO_EPSCOR/TERRACLIMATE', 
    'band':'vs', 
    'min':0, 
    'max':2923, 
    'vis':windspeed_vis}
feature_list.append(windspeed_map_info)


# TerraClimate: Monthly Climate and Climatic Water Balance for Global Terrestrial Surfaces
# -2022.12 verified
# https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
soil_moisture_info = {
    'name':'IDAHO_EPSCOR/TERRACLIMATE', 
    'band':'soil', 
    'min':0, 
    'max':8882, 
    'vis':soil_moisture_info_vis}
feature_list.append(soil_moisture_info)


# TerraClimate: Monthly Climate and Climatic Water Balance for Global Terrestrial Surfaces
# -2022.12
# https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
evapotranspiration_info = {
    'name':'IDAHO_EPSCOR/TERRACLIMATE', 
    'band':'aet', 
    'min':0, 
    'max':3140, 
    'vis':evapotranspiration_vis}
feature_list.append(evapotranspiration_info)

In [9]:
pprint(len(feature_list))

8


## Define global variables

In [10]:
MONTH_SEASON = [[4,5,6],[7,8,9],[10,11,12]]
NAME_SEASON = ['Spring', 'Summer', 'Fall']
TRAIN_RANGE = [2018, 2019, 2020]
PREDICT_RANGE = [2021]
PRED_WINDOW = 2 #2 week prior to fire map to sample features
FEATURE_COLUMN_NAMES = ['temp', 'rain', 'landcover', 'ndvi', 'cdem', 'windspeed', 'soil', 'evapotranspiration']

train_temporal_gap = 1 # 1 month

In [11]:
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
Canada = countries.filter(ee.Filter.eq('country_na', 'Canada')).geometry()
America = countries.filter(ee.Filter.eq('country_na', 'United States')).geometry()

In [12]:
# Center in Vancouver
center = [-123.1207, 49.2827]

In [13]:
PATCH_SIZE = 128
SAMPLE_SCALE = 1000
SAMPLE_POINT = 400
SHARD_NUM = 10

In [14]:
BUCKET_NAME = 'wildfire-export'
TRAINING_FOLDER = 'training_export'
TEST_FOLDER = 'test_export'
TRAINING_OUTPUT_PATH = f'gs://{BUCKET_NAME}/{TRAINING_FOLDER}/'
TEST_OUTPUT_PATH = f'gs://{BUCKET_NAME}/{TEST_FOLDER}/'
MODEL_SAVE_PATH = f'gs://{BUCKET_NAME}/model/'

In [15]:
# def process_img_value(img, map_info):
#     unmask_value = map_info['min'] - (map_info['max'] - map_info['min'])*0.1
#     return img.clamp(map_info['min'], map_info['max']).float().unmask(unmask_value)

# Data preprocessing

## Define helper functions

In [16]:
def get_range_start_from_month(year, month, gap=1):
    """
    Get a date range starting from a specified year and month and spanning a given number of months.

    Args:
    year (int): The starting year.
    month (int): The starting month (1 to 12).
    gap (int, optional): The number of months to span (default is 1).

    Returns:|
    tuple: A tuple containing the start and end dates as ee.Date objects.
    """
    start = ee.Date(f'{year}-{month:02d}-01')
    end = start.advance(gap, 'month')
    return (start, end)


def get_range_from_year(year):
    """
    Get a date range for a given year.

    Args:
    year (int): The year for which you want to obtain a date range.

    Returns:
    tuple: A tuple containing two strings representing the start and end dates of the year, e.g., ('year-01-01', 'year-12-31').
    """
    return (f'{year}-01-01', f'{year}-12-31')

In [17]:
def sum_maps(map_info, start_date, end_date):
    """
    Aggregate monthly precipitation into an annual sum.

    Args:
    map_info (dict): A dictionary containing information about the Earth Engine ImageCollection, including 'name' and 'band'.
    start_date (str): The start date for filtering images (e.g., 'YYYY-MM-DD').
    end_date (str): The end date for filtering images (e.g., 'YYYY-MM-DD').

    Returns:
    ee.Image: An Earth Engine image representing the annual sum of precipitation.

    """
    collection = ee.ImageCollection(map_info['name']).filterDate(start_date, end_date).select(map_info['band'])
    img = collection.reduce(ee.Reducer.sum()).select(map_info['band']+'_sum')
    # pprint(img.getInfo())
    return img.rename(map_info['band'])

def median_maps(map_info, start_date, end_date):
    """
    Calculate the median of maps within a specified date range.

    Args:
    map_info (dict): A dictionary containing information about the Earth Engine ImageCollection, including 'name' and 'band'.
    start_date (str): The start date for filtering images (e.g., 'YYYY-MM-DD').
    end_date (str): The end date for filtering images (e.g., 'YYYY-MM-DD').

    Returns:
    ee.Image: An Earth Engine image representing the median of maps within the specified date range.

    """
    img = ee.ImageCollection(map_info['name']).filterDate(start_date, end_date).select(map_info['band']).median()
    # pprint(img.getInfo())
    return img

In [18]:
def max_maps(map_info, start_date, end_date):
    """
    Calculate the max of maps within a specified date range.

    Args:
    map_info (dict): A dictionary containing information about the Earth Engine ImageCollection, including 'name' and 'band'.
    start_date (str): The start date for filtering images (e.g., 'YYYY-MM-DD').
    end_date (str): The end date for filtering images (e.g., 'YYYY-MM-DD').

    Returns:
    ee.Image: An Earth Engine image representing the max of maps within the specified date range.

    """
    img = ee.ImageCollection(map_info['name']).filterDate(start_date, end_date).select(map_info['band']).max()
    # pprint(img.getInfo())
    return img

In [19]:
def get_img_from_collection(map_info, start_date, end_date):
    """
    Retrieve an image from a collection based on the specified map information, start date, and end date.

    Args:
    map_info (dict): A dictionary containing information about the Earth Engine ImageCollection.
    start_date (str or ee.Date): The start date for filtering images (e.g., 'YYY-MM-DD' or an ee.Date object).
    end_date (str or ee.Date): The end date for filtering images (e.g., 'YYYY-MM-DD' or an ee.Date object).

    Returns:
    ee.Image: An Earth Engine image representing the data from the specified collection and time period.
    
    """
    if map_info.get("is_label"):
        firms = max_maps(map_info, start_date, end_date).unmask(map_info['unmask']).divide(map_info['max'])
        renamed_image = firms.ceil().byte().clamp(0, 1).rename(fire_class_band)
        return renamed_image
    
    if map_info.get('topo'):
        return ee.Image(topo_map_info['topo'])
    
    if map_info.get('no_time'):
        img = ee.ImageCollection(cdem_map_info['name']).select(cdem_map_info['band']).median()
        return img
    
    if map_info.get('annual'):
        (start_date,end_date) = get_range_from_year(start_date.get('year').getInfo())
        
    if map_info.get('sum'):
        return sum_maps(map_info, start_date, end_date).float()
        # return sum_maps(map_info, start_date, end_date).float()
        
    return median_maps(map_info, start_date, end_date).float()
    # return median_maps(map_info, start_date, end_date).float()

In [20]:
def get_input_image(start_date, end_date) -> ee.Image:
    """
    Get a combined Earth Engine image from multiple collections for a specified time period.

    Args:
    start_date (str or ee.Date): The start date for filtering images (e.g., 'YYYY-MM-DD' or an ee.Date object).
    end_date (str or ee.Date): The end date for filtering images (e.g., 'YYYY-MM-DD' or an ee.Date object).

    Returns:
    ee.Image: A combined Earth Engine image containing data from multiple collections within the specified time period.

    """
    combined_image = get_img_from_collection(feature_list[0], start_date, end_date)
    for i in range(1, len(feature_list)):
        combined_image = ee.Image.cat([combined_image, get_img_from_collection(feature_list[i], start_date, end_date)])
    return combined_image

# print(get_input_image('2021-01-01', '2021-12-31').bandNames().getInfo())

In [21]:
def get_label_image(start_date, end_date) -> ee.Image:
    return get_img_from_collection(fire_map_info, start_date, end_date)
# print(get_label_image('2021-01-01', '2021-12-31').bandNames().getInfo())

In [22]:
def test_band():
    (start_date, end_date) = get_range_start_from_month(2021, 5, 1)
    # print(start_date)
    # print()
    # print(end_date)
    
    img = get_img_from_collection(fire_map_info, start_date, end_date)
    # print(img.getInfo())
    
    # img = ee.Image(topo_map_info['topo'])
    
    # img = ee.ImageCollection(fire_map_info['name'])\
    #     .filterDate(start_date,end_date)\
    #     .select(topo_map_info['band'])\
    #     .median()
    
    print(img.getInfo())
    # print(img.getInfo())
    # print(img.bandNames().getInfo())
test_band()

{'type': 'Image', 'bands': [{'id': 'is_fire', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


## Visualize maps

In [23]:
# !jupyter trust Natural_Factors_Impacting_Carbon_Sequestration.ipynb

In [24]:
visualize_year = 2021
(visualize_start_date, visualize_end_date) = get_range_from_year(visualize_year)
(visualize_start_date, visualize_end_date)

('2021-01-01', '2021-12-31')

In [25]:
def get_tile_layer(map_info):
  if map_info.get('sum'):
    image = sum_maps(map_info, visualize_start_date, visualize_end_date)
  elif map_info.get("topo"):
    image = ee.Image(topo_map_info['topo'])
  else:
    image = median_maps(map_info, visualize_start_date, visualize_end_date)
  image=image.clip(Canada)
  vis_params = {
    'bands': [map_info['band']],
    'min': map_info['min'],
    'max': map_info['max'],
    }
  if isinstance(map_info['vis'], dict):
    vis_params['palette'] = list(map_info['vis'].values())
  else:
    vis_params['palette'] = map_info['vis']
  mapid = image.getMapId(vis_params)
  return folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=map_info['band'],
  )

# map = folium.Map(location=(center[1], center[0]), zoom_start=7)
# for map_info in feature_list:
#   get_tile_layer(map_info).add_to(map)

# get_tile_layer(fire_map_info).add_to(map)

# folium.LayerControl().add_to(map)
# map

## Export data

In [26]:
# Each season samples from training year
def get_combined_image(is_training: bool):
    """
    Retrieve training data for multiple seasons, each sampling from training years.

    Returns:
    list: A list containing seasonal data with Earth Engine images, where each season is a list of images from training years and months
    
    """
    year_range = TRAIN_RANGE if is_training else PREDICT_RANGE
    combine_image_list = [] # Store 3 elements: spring, summer, fall
    for season in MONTH_SEASON:
      season_list = [] # Store all months for this season from all training years
      for month in season:
        for year in year_range:
          time_range=get_range_start_from_month(year, month)
          input_image = get_input_image(time_range[0], time_range[1])
          labels_image = get_label_image(time_range[0].advance(PRED_WINDOW, 'week'), time_range[1].advance(PRED_WINDOW, 'week'))
          combine_image = ee.Image.cat([input_image, labels_image])
          season_list.append(combine_image)
      combine_image_list.append(season_list)
    return combine_image_list

combine_image_list = get_combined_image(True)

tdd

In [27]:
def test_fire():
    Map = geemap.Map(center=[47.295, -119.086], zoom=6)
    combine_image_list = get_combined_image(True)
    # Create an empty list to store the test results
    test_results = []

    for season in combine_image_list:
        for month_image in season:
            test_img = month_image
            t21_test = test_img.select(fire_class_band)

            region = ee.Geometry.Point([-123.1207, 49.2827]).buffer(100009)
            num_samples = 100 

            samples = t21_test.stratifiedSample(
                region=region, 
                numPoints=50,
                classBand=fire_class_band,
                scale=1000)
            sample_values = samples.aggregate_array(fire_class_band)
            lst = sample_values.getInfo()

            # contains_non_zero = any(element != 0 for element in lst)
            value_counts = {value: lst.count(value) for value in set(lst)}

            test_results.append(value_counts)        
    print(test_results)
    
# test_fire()

In [28]:
import time
import random


def sample_points(
    region: ee.Geometry, num_points: int, scale: int, is_training: bool
):
    """
    Sample points from Earth Engine images and export them to Cloud Storage in TFRecord format.

    Args:
    region (ee.Geometry): An Earth Engine geometry representing the region of interest.
    num_points (int): The total number of points to sample.
    scale (int): The scale at which to sample the points.
    """
    random.seed(int(time.time()))

    sample_shards = 10
    num_points /= sample_shards
    combine_image_list = get_combined_image(is_training)
    export_folder = TRAINING_FOLDER if is_training else TEST_FOLDER
    for i, season in enumerate(combine_image_list):
        for j, month_image in enumerate(season):
            collection = ee.FeatureCollection([])
            for k in range(sample_shards):
                fc = month_image.stratifiedSample(
                    numPoints=num_points,
                    classBand=fire_class_band,
                    region=region,
                    scale=scale,
                    seed=int(random.random() * time.time())+i+j+k
                )
                collection = collection.merge(fc)
                
            task = ee.batch.Export.table.toCloudStorage(
                collection = collection,
                description = f'{NAME_SEASON[i]}-{j}',
                bucket = BUCKET_NAME,
                fileNamePrefix = f'{export_folder}/{NAME_SEASON[i]}-{j}',
                fileFormat = 'TFRecord'
            )
            # task.start()

### Export training data

In [29]:
sample_points(Canada, SAMPLE_POINT, SAMPLE_SCALE, True)

### Export test data

In [30]:
sample_points(Canada, SAMPLE_POINT, SAMPLE_SCALE, False)

# Training

In [31]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

NUM_INPUTS = len(feature_list)
LABEL = fire_class_band
BANDS = []
for feature in feature_list:
  BANDS += [feature['band']]
FEATURE_NAMES = BANDS + [LABEL]

PATCH_SHAPE = [PATCH_SIZE, PATCH_SIZE]
columns = [tf.io.FixedLenFeature(shape=[], dtype=tf.float32) for k in FEATURE_NAMES]
# columns.append(tf.io.FixedLenFeature(shape=[], dtype=tf.int64))
FEATURES_DICT = dict(zip(FEATURE_NAMES, columns))
print(FEATURES_DICT)

BATCH_SIZE = 32

{'LST_Day_1km': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'pr': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'LC_Type1': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'NDVI': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'elevation': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'vs': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'soil': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'aet': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'is_fire': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None)}


In [32]:
fixed_range = []
for feature in feature_list:
    feature_range = [feature['min'], feature['max']]
    fixed_range.append(feature_range)
# fixed_range.append([0,1]) # after divide and byte new bands is 0, 1
normed_range = [[-1,1]]
pprint(fixed_range)

[[13000.0, 16500.0],
 [0, 2000],
 [1.0, 17.0],
 [0, 1],
 [-226, 5944],
 [0, 2923],
 [0, 8882],
 [0, 3140]]


In [33]:
def normalize_fixed(x, current_range, normed_range):
    """
    Normalize a value within a specific range to another range.

    Args:
    x: The value to be normalized.
    current_range: A list containing the minimum and maximum values of the current range.
    normed_range: A list containing the minimum and maximum values of the target range.

    Returns:
    The normalized value within the target range.
    """
    current_min, current_max = current_range[:,0], current_range[:,1]
    normed_min, normed_max = normed_range[:,0], normed_range[:,1]
    x_normed = (x - current_min) / (current_max - current_min)
    x_normed = x_normed * (normed_max - normed_min) + normed_min
    return x_normed

# normalize_fixed(np.array([5.0,5.0]),np.array([[0.0,10.0],[0.0,10.0]]), np.array([[0.0,1.0]]))

## Parse tfrecord

In [34]:
# from prettytable import PrettyTable
# import tensorflow_datasets as tfds

# def visualize_dataset(dataset):
#     dataset_len = dataset.reduce(np.int64(0), lambda x, _: x + 1).numpy()
#     print(f'dataset len: {dataset_len}')
#     ds_numpy = tfds.as_numpy(dataset)
#     x = PrettyTable()
#     # x.field_names = [key for key in FEATURE_NAMES]
#     for row in ds_numpy:
#         x.add_row(row)
#     print(x)

In [35]:
def test_balance_dataset(dataset):
    cnt_0 = 0
    cnt_1 = 0
    for inputs, label in dataset.as_numpy_iterator():
        cnt_0 += (1 if label == 0 else 0)
        cnt_1 += (1 if label == 1 else 0)
    
    print(f'cnt 0={cnt_0}, cnt_1={cnt_1}')

In [36]:
def parse_tfrecord(example_proto):
    """The parsing function.
    Read a serialized example into the structure defined by FEATURES_DICT.
    Args:
    example_proto: a serialized Example.
    Returns:
    A dictionary of tensors, keyed by feature name.
    """
    parsed_features = tf.io.parse_single_example(example_proto, FEATURES_DICT)
    # print(f'Parsed features: {parsed_features}')
    return parsed_features

def to_tuple(inputs, deb=True):
    """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
    Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
    Args:
    inputs: A dictionary of tensors, keyed by feature name.
    Returns:
    A tuple of (inputs, outputs).
    """
    feature_list = [inputs.get(key) for key in BANDS]
    feature_stacked = tf.stack(feature_list, axis=0)

    # Convert from CHW to HWC
    # See https://caffe2.ai/docs/tutorial-image-pre-processing.html
    # feature_stacked = tf.transpose(feature_stacked, [1, 2, 0])
    feature_stacked = normalize_fixed(feature_stacked, np.array(fixed_range, dtype="float32"), np.array(normed_range, dtype="float32"))
    
    label = inputs.get(LABEL)
    label = 1 if label > 0.5 else 0
    # label_list = tf.transpose(label_list, [1, 2, 0])
    
    return feature_stacked, label

def get_dataset(pattern):
    """Function to read, parse and format to tuple a set of input tfrecord files.
    Get all the files matching the pattern, parse and convert to tuple.
    Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
    Returns:
    A tf.data.Dataset
    """
    print(pattern)
    glob = tf.io.gfile.glob(pattern)
    dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
    dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
    dataset = dataset.map(to_tuple, num_parallel_calls=5)
    return dataset

In [37]:
dataset_spring = get_dataset(f'{TRAINING_OUTPUT_PATH}Spring*')
# test_balance_dataset(dataset_spring)

gs://wildfire-export/training_export/Spring*


2023-11-28 15:16:32.294709: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


AbortedError: All 10 retry attempts failed. The last failure: Error executing an HTTP request: libcurl code 60 meaning 'SSL peer certificate or SSH remote key was not OK', error details: SSL certificate problem: unable to get local issuer certificate
	 when reading gs://wildfire-export/training_export

In [ ]:
dataset_summer = get_dataset(f'{TRAINING_OUTPUT_PATH}Summer*')
dataset_fall = get_dataset(f'{TRAINING_OUTPUT_PATH}Fall*')
# test_balance_dataset(dataset_summer)
# test_balance_dataset(dataset_fall)

In [ ]:
TRAIN_EVAL_RATIO = 80  # percent for training, the rest for testing

def split_dataset(dataset: tf.data.Dataset, batch_size) -> tuple[tf.data.Dataset, tf.data.Dataset]:
    """
    Splits a TensorFlow dataset into training and evaluation datasets.

    Args:
        dataset (tf.data.Dataset): The input dataset to be split.
        batch_size (int): The batch size for creating batches of data.

    Returns:
        tuple[tf.data.Dataset, tf.data.Dataset]: A tuple containing the training and evaluation datasets.
    """
    dataset_shuffle = dataset.shuffle(1)
    indexed_dataset = dataset_shuffle.enumerate()  # add an index to each example
    train_dataset = (
        indexed_dataset.filter(lambda i, _: i % 100 <= TRAIN_EVAL_RATIO)
        .map(lambda _, data: data)  # remove index
        .shuffle(batch_size * 8)  # randomize the examples for the batches
        .batch(batch_size)  # batch randomized examples
    )
    eval_dataset = (
        indexed_dataset.filter(lambda i, _: i % 100 > TRAIN_EVAL_RATIO)
        .map(lambda _, data: data, num_parallel_calls=tf.data.AUTOTUNE)  # remove index
        .batch(batch_size)  # batch the parsed examples, no need to shuffle
    )
    return (train_dataset, eval_dataset)

### Balance sanity check

In [ ]:
def verify_train_eval_label(dataset, dataset_name):
    num_0 = 0
    num_1 = 0
    for element in dataset.as_numpy_iterator():
        num_1 += np.count_nonzero(element[1])
        num_0 += len(element[1]) - np.count_nonzero(element[1])
        
    print(f'{dataset_name} count_0 = {num_0}, count_1 = {num_1}')

In [ ]:
dataset_all_seasons = [dataset_spring, dataset_summer, dataset_fall]
train_dataset_list = []
eval_dataset_list = []

for dataset in dataset_all_seasons:
    (train_season, eval_season) = split_dataset(dataset, batch_size=32)
    train_dataset_list.append(train_season)    
    eval_dataset_list.append(eval_season)

In [ ]:
def balance_sanity_check():
    for train_season, eval_season in zip(train_dataset_list, eval_dataset_list):
        verify_train_eval_label(train_season, 'train')
        verify_train_eval_label(eval_season, 'eval')
# balance_sanity_check()

### Convert dataset to dataframe for ML

In [ ]:
import pandas as pd

def convert_dataset_to_dataframe(dataset):
    """
    Converts a TensorFlow dataset to a Pandas DataFrame.

    Args:
    dataset (tf.data.Dataset): A TensorFlow dataset containing feature-label pairs.

    Returns:
    pd.DataFrame: A Pandas DataFrame with feature and label columns. 
    """
    data_dict = {'features': [], 'labels': []}
    
    for x, y in dataset:
        features = x.numpy()
        labels = y.numpy()
        
        data_dict['features'].extend(features)
        data_dict['labels'].extend(labels)
    
    df = pd.DataFrame(data_dict)
    
    labels_df = df['labels']
    df = pd.DataFrame(df['features'].to_list(), columns=FEATURE_COLUMN_NAMES)
    df['labels'] = labels_df                              
    
    return df


train_df = [convert_dataset_to_dataframe(dataset) for dataset in train_dataset_list]
eval_df = [convert_dataset_to_dataframe(dataset) for dataset in eval_dataset_list]

In [ ]:
train_df[-1]['labels']

## Define plot metrics

In [ ]:
import matplotlib.pyplot as plt

def plot_metrics(history, metric_name, plot_title):
    plt.figure(figsize=(6, 4))
    plt.plot(history.history[metric_name], label=f'training {metric_name}')
    plt.plot(history.history[f'val_{metric_name}'], label=f'validation {metric_name}')
    plt.xlabel('Epoch')
    plt.ylabel(f'{metric_name}')
    plt.title(f'Training and Validation {plot_title} {metric_name}')
    plt.legend()
    plt.grid(True)
    plt.show()

## helper function to save and load model

In [ ]:
SEASONS = ["Spring", "Summer", "Fall"]

def save_model_ann(model_list, model_name):
    for i, model in enumerate(model_list):
        model.save(f"{MODEL_SAVE_PATH}{model_name}/{NAME_SEASON[i]}")
        
def load_model_ann(model_name, seasons):
    loaded_models = []

    for season in seasons:
        print(f"loading {season}")
        model_path = f"{MODEL_SAVE_PATH}{model_name}/{season}"
        loaded_model = keras.models.load_model(model_path)
        loaded_models.append(loaded_model)

    return loaded_models

## helper function to save and load metrics

In [ ]:
import pickle

def save_metrics(metrics, filename):
    with open(filename, "wb") as file:
        pickle.dump(metrics, file)

def load_metrics(filename):
    with open(filename, "rb") as file:
        return pickle.load(file)

In [ ]:
stop

# Model fitting

## Mia - ANN


In [ ]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.losses as losses
import tensorflow.keras.models as models
import tensorflow.keras.metrics as metrics
import tensorflow.keras.optimizers as optimizers

In [ ]:
all_metric_names = ['Accuracy', 'Recall', 'AUC', 'FalseNegatives']
def plot_model_history(history, plot_title):
    for metric_name in all_metric_names:
        plot_metrics(history, metric_name, plot_title)

In [ ]:
def train_ann_model(model, train_dataset, eval_dataset, epochs = 10):
    # Compile Model
    model.compile(optimizer = 'adam', metrics=[
        tf.keras.metrics.BinaryAccuracy(name=all_metric_names[0]), 
        tf.keras.metrics.Recall(name=all_metric_names[1]),
        tf.keras.metrics.AUC(name=all_metric_names[2]),
        tf.keras.metrics.FalseNegatives(name=all_metric_names[3])], 
        loss ='binary_crossentropy',
        )
    # Train Model
    history = model.fit(train_dataset, validation_data = eval_dataset, epochs = epochs)
    return model, history

### ANN baseline model

In [ ]:
def get_ann_baseline_model():
  model = tf.keras.Sequential()
  # input layer + 1st hidden layer
  model.add(layers.Dense(32, input_dim=NUM_INPUTS, activation='relu', name='Dense1'))
  # hidden layer
  model.add(layers.Dense(16, activation='relu', name='Dense2'))
  # output layer
  model.add(layers.Dense(1, activation = 'sigmoid', name='Output'))
  model.summary()
  tf.keras.utils.plot_model(model, show_shapes=True)
  return model

In [ ]:
model_ann_baseline_list = []
for i, (train_dataset, eval_dataset) in enumerate(zip(train_dataset_list, eval_dataset_list)):
    tf.keras.backend.clear_session()
    model_ann_baseline, history_baseline = train_ann_model(get_ann_baseline_model(), train_dataset, eval_dataset)
    model_ann_baseline_list.append(model_ann_baseline)
    plot_model_history(history_baseline, NAME_SEASON[i])
    # break

In [ ]:
save_model_ann(model_ann_baseline_list, "model_ann_baseline")

### ANN deeper model

In [ ]:
def get_ann_deeper_model():
    model = tf.keras.Sequential()
    init_filter = 128
    cur_filter = init_filter/2
    cur_layer = 1
    # input layer + 1st hidden layer
    model.add(layers.Dense(init_filter, input_dim=NUM_INPUTS, activation='relu', name='Dense0'))
    # hidden layer
    while(cur_layer < 5):
        model.add(layers.Dense(cur_filter, activation='relu', name=f'Dense{cur_layer}'))
        model.add(layers.Dropout(0.2))
        model.add(layers.BatchNormalization())
        # cur_filter /= 2
        cur_layer += 1
    # output layer
    model.add(layers.Dense(1, activation = 'sigmoid', name='Output'))
    model.summary()
    tf.keras.utils.plot_model(model, show_shapes=True)
    return model

# get_ann_deeper_model()

In [ ]:
model_ann_deep_list = []
for i, (train_dataset, eval_dataset) in enumerate(zip(train_dataset_list, eval_dataset_list)):
    tf.keras.backend.clear_session()
    model_ann_deep, history_ann_deep = train_ann_model(get_ann_deeper_model(), train_dataset, eval_dataset, 10)
    model_ann_deep_list.append(model_ann_deep)
    plot_model_history(history_ann_deep, NAME_SEASON[i])
    # break

In [ ]:
save_model_ann(model_ann_deep_list, "model_ann_deep")

### TabTransformer https://keras.io/examples/structured_data/tabtransformer/

In [ ]:
# !pip install pydot[and-cuda] --user
# !pip install graphviz[and-cuda] --user

In [ ]:
# def run_experiment(
#     model,
#     train_dataset,
#     validation_dataset,
#     num_epochs,
#     learning_rate,
#     weight_decay,
# ):

#     optimizer = optimizers.AdamW(
#         learning_rate=learning_rate, weight_decay=weight_decay
#     )

#     model.compile(
#         optimizer=optimizer,
#         loss=keras.losses.BinaryCrossentropy(),
#         metrics=[keras.metrics.BinaryAccuracy(name="accuracy")],
#     )

#     print("Start training the model...")
#     history = model.fit(
#         train_dataset, epochs=num_epochs, validation_data=validation_dataset
#     )
#     print("Model training finished")

#     _, accuracy = model.evaluate(validation_dataset, verbose=0)

#     print(f"Validation accuracy: {round(accuracy * 100, 2)}%")

#     return history

In [ ]:
# def create_model_inputs():
#     inputs = {}
#     for feature_name in BANDS:
#         inputs[feature_name] = layers.Input(
#                     name=feature_name, shape=(), dtype=tf.float32)   
#     return inputs

In [ ]:
# def encode_inputs(inputs):
#     encoded_categorical_feature_list = []
#     numerical_feature_list = []

#     for feature_name in inputs:
#         # Use the numerical features as-is.
#         numerical_feature = tf.expand_dims(inputs[feature_name], -1)
#         numerical_feature_list.append(numerical_feature)

#     return layers.concatenate(
#         encoded_categorical_feature_list + numerical_feature_list
#     )

In [ ]:
# def create_mlp(hidden_units, dropout_rate, activation, normalization_layer, name=None):
#     print(f'hidden_units={hidden_units} ')
#     mlp_layers = []
#     for units in hidden_units:
#         mlp_layers.append(normalization_layer),
#         mlp_layers.append(layers.Dense(units, activation=activation))
#         mlp_layers.append(layers.Dropout(dropout_rate))

#     return keras.Sequential(mlp_layers, name=name)

In [ ]:
# LEARNING_RATE = 0.001
# WEIGHT_DECAY = 0.0001
# DROPOUT_RATE = 0.2
# NUM_EPOCHS = 10

# NUM_TRANSFORMER_BLOCKS = 3  # Number of transformer blocks.
# NUM_HEADS = 4  # Number of attention heads.
# EMBEDDING_DIMS = 16  # Embedding dimensions of the categorical features.
# MLP_HIDDEN_UNITS_FACTORS = [
#     2,
#     1,
# ]  # MLP hidden layer units, as factors of the number of inputs.
# NUM_MLP_BLOCKS = 2  # Number of MLP blocks in the baseline model.

#### Baseline model

In [ ]:
# def create_baseline_model(
#     num_mlp_blocks, mlp_hidden_units_factors, dropout_rate
# ):

#     # Create model inputs.
#     inputs = create_model_inputs()
#     # encode features.
#     features = encode_inputs(
#         inputs
#     )
#     # Compute Feedforward layer units.
#     feedforward_units = [features.shape[-1]]
#     print(features.shape)
#     # Create several feedforwad layers with skip connections.
#     for layer_idx in range(num_mlp_blocks):
#         features = create_mlp(
#             hidden_units=feedforward_units,
#             dropout_rate=dropout_rate,
#             activation=keras.activations.gelu,
#             normalization_layer=layers.LayerNormalization(epsilon=1e-6),
#             name=f"feedforward_{layer_idx}",
#         )(features)

#     # Compute MLP hidden_units.
#     mlp_hidden_units = [
#         factor * features.shape[-1] for factor in mlp_hidden_units_factors
#     ]
#     # Create final MLP.
#     features = create_mlp(
#         hidden_units=mlp_hidden_units,
#         dropout_rate=dropout_rate,
#         activation=keras.activations.selu,
#         normalization_layer=layers.BatchNormalization(),
#         name="MLP",
#     )(features)

#     # Add a sigmoid as a binary classifer.
#     outputs = layers.Dense(units=1, activation="sigmoid", name="sigmoid")(features)
#     model = keras.Model(inputs=inputs, outputs=outputs)
#     return model

In [ ]:
# transformer_baseline_model = create_baseline_model(
#     num_mlp_blocks=NUM_MLP_BLOCKS,
#     mlp_hidden_units_factors=MLP_HIDDEN_UNITS_FACTORS,
#     dropout_rate=DROPOUT_RATE,
# )

# print("Total model weights:", transformer_baseline_model.count_params())
# keras.utils.plot_model(transformer_baseline_model, show_shapes=True, rankdir="LR")

In [ ]:
# history_transformer_baseline = run_experiment(
#     model=transformer_baseline_model,
#     train_dataset=train_dataset_list[0],
#     validation_dataset=eval_dataset_list[0],
#     num_epochs=NUM_EPOCHS,
#     learning_rate=LEARNING_RATE,
#     weight_decay=WEIGHT_DECAY,
# )

## Huizi - RF

1. Data Preparation:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
# Create a dictionary to store the metrics for all seasons
all_metrics = {
    "Season": [],
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1-Score": [],
    "AUC": []
}
SEASONS = ["Spring", "Summer", "Fall"]

# Compute metrics for each season
for season, train_data, eval_data in zip(SEASONS, train_df, eval_df):
    X_train = train_data.drop(columns="labels")
    y_train = train_data["labels"]
    X_eval = eval_data.drop(columns="labels")
    y_eval = eval_data["labels"]

    # Training Random Forest model
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    
    y_pred = rf.predict(X_eval)
    y_pred_proba = rf.predict_proba(X_eval)[:, 1]  # assuming binary classification

    # Computing metrics
    accuracy = accuracy_score(y_eval, y_pred)
    precision = precision_score(y_eval, y_pred, average='macro')
    recall = recall_score(y_eval, y_pred, average='macro')
    f1 = f1_score(y_eval, y_pred, average='macro')
    auc = roc_auc_score(y_eval, y_pred_proba)

    # Storing metrics
    all_metrics["Season"].append(season)
    all_metrics["Accuracy"].append(accuracy)
    all_metrics["Precision"].append(precision)
    all_metrics["Recall"].append(recall)
    all_metrics["F1-Score"].append(f1)
    all_metrics["AUC"].append(auc)

# Convert metrics to DataFrame for presentation
metrics_table = pd.DataFrame(all_metrics)
metrics_table

In [ ]:
# Visualization
plt.figure(figsize=(12, 8))

# Seasons for x-axis
x = metrics_table['Season']

# Plotting each metric as a line
for metric in ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC']:
    plt.plot(x, metrics_table[metric], marker='o', label=metric)

# Setting the title, labels, and legend
plt.xlabel('Season')
plt.ylabel('Score')
plt.title('Performance Metrics Across Seasons')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend()

plt.show()

In [ ]:
# 1. Feature Importance
from sklearn.tree import plot_tree
import itertools
from sklearn.preprocessing import label_binarize

SEASONS = ["Spring", "Summer", "Fall"]
season_feature_importances = {}
rf_models = []

for season, train_data in zip(SEASONS, train_df):
    X_train = train_data.drop(columns="labels")
    y_train = train_data["labels"]

    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    rf_models.append(rf)
    
    # Extract feature importance
    season_feature_importances[season] = rf.feature_importances_

# Convert to DataFrame for easier plotting
feature_importance_df = pd.DataFrame(season_feature_importances, index=X_train.columns)

for season in SEASONS:
    plt.figure(figsize=(10, 6))
    feature_importance_df[season].sort_values().plot(kind='barh', title=f'Feature Importance for {season}')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

# Plotting
feature_importance_df.plot(kind='bar', figsize=(15, 7))
plt.title('Feature Importance across Seasons')
plt.ylabel('Importance')
plt.xlabel('Features')
plt.legend(title='Season')
plt.tight_layout()
plt.show()

2. Hyperparameter Tuning:

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter distributions rather than a fixed set of values per hyperparameter
param_dist = {
    'n_estimators': [10, 50, 100, 200, 500],
    'max_depth': [None, 10, 15, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

best_params_per_season = {}

# # Hyperparameter tuning for each season using RandomizedSearchCV
# for season, train_data in zip(SEASONS, train_df):
#     X_train = train_data.drop(columns="labels")
#     y_train = train_data["labels"]
    
#     rf = RandomForestClassifier()
#     random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, 
#                                        n_iter=50, cv=3, n_jobs=-1, verbose=2, scoring='accuracy', random_state=42)
#     random_search.fit(X_train, y_train)
    
#     best_params_per_season[season] = random_search.best_params_

In [ ]:
best_params_per_season = {'Spring': {'n_estimators': 100,
  'min_samples_split': 2,
  'min_samples_leaf': 1,
  'max_depth': 20,
  'bootstrap': True},
 'Summer': {'n_estimators': 200,
  'min_samples_split': 5,
  'min_samples_leaf': 2,
  'max_depth': 15,
  'bootstrap': True},
 'Fall': {'n_estimators': 50,
  'min_samples_split': 2,
  'min_samples_leaf': 2,
  'max_depth': 10,
  'bootstrap': False}}

In [ ]:
# Train and evaluate models using the best parameters for each season
tuned_all_metrics = {
    "Season": [],
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1-Score": [],
    "AUC": []
}

tuned_feature_importances = {}

for season, train_data, eval_data in zip(SEASONS, train_df, eval_df):
    X_train = train_data.drop(columns="labels")
    y_train = train_data["labels"]
    X_eval = eval_data.drop(columns="labels")
    y_eval = eval_data["labels"]
    
    rf = RandomForestClassifier(**best_params_per_season[season])
    rf.fit(X_train, y_train)
    
    y_pred = rf.predict(X_eval)
    y_pred_proba = rf.predict_proba(X_eval)[:, 1]
    
    accuracy = accuracy_score(y_eval, y_pred)
    precision = precision_score(y_eval, y_pred, average='macro')
    recall = recall_score(y_eval, y_pred, average='macro')
    f1 = f1_score(y_eval, y_pred, average='macro')
    auc = roc_auc_score(y_eval, y_pred_proba)
    
    tuned_all_metrics["Season"].append(season)
    tuned_all_metrics["Accuracy"].append(accuracy)
    tuned_all_metrics["Precision"].append(precision)
    tuned_all_metrics["Recall"].append(recall)
    tuned_all_metrics["F1-Score"].append(f1)
    tuned_all_metrics["AUC"].append(auc)

    # Store feature importances
    tuned_feature_importances[season] = rf.feature_importances_

# Display metrics
tuned_metrics_table = pd.DataFrame(tuned_all_metrics)
print(tuned_metrics_table)

In [ ]:
# Plotting the performance metrics
plt.figure(figsize=(12, 8))
x = tuned_metrics_table['Season']
for metric in ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC']:
    plt.plot(x, tuned_metrics_table[metric], marker='o', label=metric)
plt.xlabel('Season')
plt.ylabel('Score')
plt.title('Tuned Performance Metrics Across Seasons')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend()
plt.show()

In [ ]:
# Display feature importances after tuning
tuned_feature_importance_df = pd.DataFrame(tuned_feature_importances, index=X_train.columns)
for season in SEASONS:
    plt.figure(figsize=(10, 6))
    tuned_feature_importance_df[season].sort_values().plot(kind='barh', title=f'Tuned Feature Importance for {season}')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

In [ ]:
# Plotting all together
tuned_feature_importance_df.plot(kind='bar', figsize=(15, 7))
plt.title('Tuned Feature Importance across Seasons')
plt.ylabel('Importance')
plt.xlabel('Features')
plt.legend(title='Season')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))

# Seasons for x-axis
x = metrics_table['Season']

# Plotting each metric as a line for the original model
for metric in ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC']:
    plt.plot(x, metrics_table[metric], marker='o', linestyle='-', label=f'Original {metric}')

# Plotting each metric as a line for the tuned model
for metric in ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC']:
    plt.plot(x, tuned_metrics_table[metric], marker='x', linestyle='--', label=f'Tuned {metric}')

# Setting the title, labels, and legend
plt.xlabel('Season')
plt.ylabel('Score')
plt.title('Comparison of Original vs. Tuned Performance Metrics Across Seasons')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend()

plt.show()

In [ ]:
# 2. Tree Visualization (we'll visualize just one tree for simplicity)

# Initialize list to store tree depths for each season
tree_depths = {season: [] for season in SEASONS}

# Extract tree depths for each season
for rf, season in zip(rf_models, SEASONS):
    depths = [estimator.tree_.max_depth for estimator in rf.estimators_]
    tree_depths[season] = depths

# Plotting tree depth distribution
for season, depths in tree_depths.items():
    plt.figure(figsize=(10, 6))
    plt.hist(depths, bins=20, edgecolor="k", alpha=0.7)
    plt.title(f'Tree Depth Distribution for {season} Season')
    plt.xlabel('Tree Depth')
    plt.ylabel('Number of Trees')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

features = X_train.columns.tolist()

for rf, season in zip(rf_models, SEASONS):
    plt.figure(figsize=(20, 10))
    plot_tree(rf.estimators_[0], filled=True, feature_names=features, class_names=True, max_depth=3)
    plt.title(f'One of the Trees from the Random Forest for {season}')
    plt.show()

In [ ]:
# 3. OOB Error Rate For Each Season
oob_errors = []

# Model Training and OOB computation
for season, train_data in zip(SEASONS, train_df):
    X_train = train_data.drop(columns="labels")
    y_train = train_data["labels"]

    rf = RandomForestClassifier(oob_score=True)
    rf.fit(X_train, y_train)
    
    oob_error = 1 - rf.oob_score_
    oob_errors.append(oob_error)

# Displaying the OOB errors for each season
oob_error_df = pd.DataFrame({
    'Season': SEASONS,
    'OOB Error Rate': oob_errors
})

oob_error_df

In [ ]:
# 4. Confusion Matrix
# Function to plot confusion matrix

from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

# Plot confusion matrix for each season
for rf, eval_data, season in zip(rf_models, eval_df, SEASONS):
    X_eval = eval_data.drop(columns="labels")
    y_eval = eval_data["labels"]
    y_pred = rf.predict(X_eval)

    # Compute confusion matrix
    cnf_matrix = confusion_matrix(y_eval, y_pred)

    # Plot confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=["Class 0", "Class 1"], title=f'Confusion Matrix for {season} Season')
    plt.show()


In [ ]:
# 5. Class Prediction Probabilities

y_pred_probas = []
for rf, eval_data in zip(rf_models, eval_df):
    X_eval = eval_data.drop(columns="labels")
    y_pred_proba = rf.predict_proba(X_eval)
    y_pred_probas.append(y_pred_proba)

# Plot class prediction probabilities for each season
for y_pred_proba, season in zip(y_pred_probas, SEASONS):
    # Get the number of predicted classes
    n_classes = y_pred_proba.shape[1]

    plt.figure(figsize=(8, 6))
    for i in range(n_classes):
        plt.hist(y_pred_proba[:, i], bins=50, alpha=0.5, label=f'Class {i}')
    plt.title(f'Class Prediction Probabilities for {season} Season')
    plt.xlabel('Probability')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()

In [ ]:
# 6. ROC Curve and AUC
from sklearn.preprocessing import label_binarize
from sklearn.metrics import auc

# Binarily encode the labels for each season
y_bin_evals = [label_binarize(y, classes=[0, 1, 2]) for y in [eval_data["labels"] for eval_data in eval_df]]

# Plot ROC and compute AUC for each season
for y_bin_eval, y_pred_proba, season in zip(y_bin_evals, y_pred_probas, SEASONS):
    num_classes = y_pred_proba.shape[1]
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(y_bin_eval[:, i], y_pred_proba[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    plt.figure(figsize=(8, 6))
    for i in range(num_classes):
        plt.plot(fpr[i], tpr[i], label=f'Class {i} (area = {roc_auc[i]:.2f})')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for {season} Season')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
# 7. Precision-Recall Curve
for idx, season in enumerate(SEASONS):
    X_eval = eval_df[idx].drop(columns="labels")
    y_eval = eval_df[idx]["labels"]

    # Using the previously trained random forest models
    rf_model = rf_models[idx]

    y_pred_proba = rf_model.predict_proba(X_eval)
    unique_classes = np.unique(y_eval)
    y_bin_eval = label_binarize(y_eval, classes=unique_classes)

    precision = dict()
    recall = dict()
    
    n_classes = y_pred_proba.shape[1]
    
    plt.figure(figsize=(8, 6))
    for i in range(n_classes):
        if n_classes == 1:
            # Handle binary case
            precision[i], recall[i], _ = precision_recall_curve(y_bin_eval.ravel(), y_pred_proba[:, i])
            plt.plot(recall[i], precision[i], lw=2, label=f'Precision-Recall curve of class {unique_classes[i]}')
        else:
            precision[i], recall[i], _ = precision_recall_curve(y_bin_eval[:, i], y_pred_proba[:, i])
            plt.plot(recall[i], precision[i], lw=2, label=f'Precision-Recall curve of class {unique_classes[i]}')
    
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve for {season} Season')
    plt.legend(loc="best")
    plt.show()


In [ ]:
# 8. Box Plot
# Plotting box plots for each season separately
for season, season_data in zip(SEASONS, train_df):
    plt.figure(figsize=(15, 8))
    season_data.boxplot()
    plt.title(f'Feature Distribution in {season} Season')
    plt.ylabel('Value')
    plt.xticks(rotation=45)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.show()

## Justin

In [ ]:
import joblib

def save_models_joblib(model_list, prefix):
    for i, model in enumerate(model_list):
        model_filename = f"{prefix}_{NAME_SEASON[i]}.pkl"
        joblib.dump(model, model_filename)
        
def load_models_joblib(prefix, num_models=3):
    loaded_models = []
    for i in range(num_models):
        model_filename = f"{prefix}_{NAME_SEASON[i]}.pkl"
        loaded_model = joblib.load(model_filename)
        loaded_models.append(loaded_model)
    return loaded_models

In [ ]:
SEASONS = ["Spring", "Summer", "Fall"]
for season, train_data, eval_data in zip(SEASONS, train_df, eval_df):
    X_train = train_data.drop(columns="labels")
    y_train = train_data["labels"]
    X_eval = eval_data.drop(columns="labels")
    y_eval = eval_data["labels"]

### improved ANN

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential

def get_ann2_model():
    model = Sequential()
    
    model.add(Dense(64, input_dim=NUM_INPUTS, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    
    return model

In [ ]:
model_ann2_list = []
for i, (train_dataset, eval_dataset) in enumerate(zip(train_dataset_list, eval_dataset_list)):
    model_ann2, history_ann2 = train_ann_model(get_ann2_model(), train_dataset, eval_dataset)
    model_ann2_list.append(model_ann2)
    plot_model_history(history_ann2, NAME_SEASON[i])

In [ ]:
for i, model in enumerate(model_ann_baseline_list):
    model.save(f"{MODEL_SAVE_PATH}model_ann2/{NAME_SEASON[i]}")

## xgboost

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, precision_score
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import numpy as np

In [ ]:
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 11, 111],
    'learning_rate': [0.1, 0.01, 0.001]
}

best_models_xgb = []


season_feature_importances_xgb = {}

for i, (train_dataframe, eval_dataframe) in enumerate(zip(train_df, eval_df)):
    train_dataframe = pd.get_dummies(train_dataframe, columns=['labels'])
    eval_dataframe = pd.get_dummies(eval_dataframe, columns=['labels'])

    train_dataframe_X = train_dataframe.loc[:, ~train_dataframe.columns.str.startswith('labels_')]
    eval_dataframe_X = eval_dataframe.loc[:, ~eval_dataframe.columns.str.startswith('labels_')]
    
    train_dataframe_Y = train_dataframe.loc[:, train_dataframe.columns.str.startswith('labels_')]
    eval_dataframe_Y = eval_dataframe.loc[:, eval_dataframe.columns.str.startswith('labels_')]

    model = xgb.XGBClassifier()
    
    n_iter_search = 10  
    random_search = RandomizedSearchCV(
        model, 
        param_distributions=param_dist, 
        n_iter=n_iter_search, 
        cv=3, 
        n_jobs=-1, 
        verbose=1, 
        scoring='accuracy')
    random_search.fit(train_dataframe_X, train_dataframe_Y)
    best_model = random_search.best_estimator_

    model.fit(train_dataframe_X, train_dataframe_Y)

    eval_predictions = model.predict(eval_dataframe_X)
    
    accuracy = accuracy_score(eval_dataframe_Y, eval_predictions)
    recall = recall_score(eval_dataframe_Y, eval_predictions, average='macro')

    # The confusion matrix takes a vector of labels (not the one-hot encoding)
    conf_matrix = confusion_matrix(eval_dataframe_Y.values.argmax(axis=1), eval_predictions.argmax(axis=1))
    feature_importance = model.feature_importances_
    
    print(f"Model {i + 1} - Accuracy: {accuracy}")
    print(f"Model {i + 1} - Recall: {recall}")
    print(f"Model {i + 1} - Confusion Matrix:\n{conf_matrix}")
    
    season_feature_importances_xgb[f'Model {i + 1}'] = feature_importance

    best_models_xgb.append(best_model)

save best models

In [ ]:
for i, model in enumerate(best_models_xgb):
    model.save_model(f'xgb_model_{i + 1}.json')

load models

In [ ]:
best_models_xgb=[]
loaded_model1 = xgb.Booster(model_file='xgb_model_1.json')
loaded_model2 = xgb.Booster(model_file='xgb_model_2.json')
loaded_model3 = xgb.Booster(model_file='xgb_model_3.json')
best_models_xgb.append(loaded_model1)
best_models_xgb.append(loaded_model2)
best_models_xgb.append(loaded_model3)
best_models_xgb

In [ ]:
for model_num, importances in season_feature_importances_xgb.items():
    print(f"Feature Importances for {model_num}")
    for feature, importance in zip(train_dataframe_X.columns, importances):
        print(f"{feature}: {importance}")
    print("\n")

In [ ]:
feature_importance_df_xgb = pd.DataFrame(season_feature_importances_xgb, index=X_train.columns)

for i, (season,train_dataframe, eval_dataframe) in enumerate(zip(SEASONS, train_df, eval_df)):
    plt.figure(figsize=(10, 6))
    feature_importance_df_xgb[f'Model {i + 1}'].sort_values().plot(kind='barh', title=f'Feature Importance for {season}')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

feature_importance_df_xgb.plot(kind='bar', figsize=(15, 7))
plt.title('Feature Importance across Seasons')
plt.ylabel('Importance')
plt.xlabel('Features')
plt.legend(title='Season')
plt.tight_layout()
plt.show()

- Kernel SVM

In [ ]:
for season, train_data, eval_data in zip(SEASONS, train_df, eval_df):
    X_train = train_data.drop(columns="labels")
    y_train = train_data["labels"]
    X_eval = eval_data.drop(columns="labels")
    y_eval = eval_data["labels"]

X_train

In [ ]:
from sklearn.svm import SVC


from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import numpy as np

param_dist = {
    'C': [0.01, 0.1, 1, 10],
    'kernel': ['rbf', 'sigmoid', 'poly'],
    'gamma': [0.001, 0.01, 0.1, 1, 'scale', 'auto'],
    'degree': [2, 3, 4, 5],
}

best_models_svm = []

SEASONS = ["Spring", "Summer", "Fall"]

for i, (train_data, eval_data) in enumerate(zip(train_df, eval_df)):
    svm = SVC()
    
    n_iter_search = 10
    random_search = RandomizedSearchCV(svm, param_distributions=param_dist, n_iter=n_iter_search, cv=3, n_jobs=-1, verbose=1, scoring='accuracy')
    random_search.fit(X_train, y_train)
    best_svm = random_search.best_estimator_
    
    y_pred = best_svm.predict(X_eval)
    
    accuracy = accuracy_score(y_eval, y_pred)
    recall = recall_score(y_eval, y_pred, average='macro')
    f1 = f1_score(y_eval, y_pred, average='macro')
    conf_matrix = confusion_matrix(y_eval, y_pred)
    
    print(f"Model {i + 1} - Accuracy: {accuracy}")
    print(f"Model {i + 1} - Recall: {recall}")
    print(f"Model {i + 1} - F1-Score: {f1}")
    print(f"Model {i + 1} - Confusion Matrix:\n{conf_matrix}")
    
    best_models_svm.append(best_svm)

In [ ]:
import joblib

joblib.dump(best_models_svm, 'svm_models.pkl')

In [ ]:
svm_loaded_models = joblib.load('svm_models.pkl')
svm_loaded_models

- lstm

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from tensorflow import keras
import numpy as np

num_epochs=10
SEASONS = ["Spring", "Summer", "Fall"]

def create_lstm_model(hidden_units=64, lstm_layers=1, dense_units=32, dropout_rate=0.0):
    model = keras.Sequential()
    model.add(keras.layers.LSTM(hidden_units, input_shape=(1, num_features), return_sequences=True))
    
    for _ in range(lstm_layers - 1):
        model.add(keras.layers.LSTM(hidden_units, return_sequences=True))
    
    model.add(keras.layers.LSTM(hidden_units, return_sequences=False))
    model.add(keras.layers.Dense(dense_units, activation='relu'))
    model.add(keras.layers.Dropout(dropout_rate))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', 
                  metrics=[keras.metrics.BinaryAccuracy(name='Accuracy'), 
                           keras.metrics.Recall(name='Recall'),
                          ])
    return model

param_dist = {
    'hidden_units': np.arange(32, 128, 32),
    'lstm_layers': np.arange(1, 4),
    'dense_units': np.arange(16, 64, 16),
    'dropout_rate': [0.0, 0.2, 0.4, 0.6, 0.8],
}

all_metrics_lstm = {
    "Season": [],
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1-Score": [],
}

lstm_best_models = []

for season, train_data, eval_data in zip(SEASONS, train_df, eval_df):
    X_train = np.array(train_data.drop(columns="labels"))
    y_train = np.array(train_data["labels"])
    X_eval = np.array(eval_data.drop(columns="labels"))
    y_eval = np.array(eval_data["labels"])

    num_time_steps, num_features = X_train.shape
    X_train = X_train.reshape(num_time_steps, 1, num_features)
    num_time_steps, num_features = X_eval.shape
    X_eval = X_eval.reshape(num_time_steps, 1, num_features)

    best_accuracy = 0
    best_model = None

    for _ in range(10): 
        hidden_units = np.random.choice(param_dist['hidden_units'])
        lstm_layers = np.random.choice(param_dist['lstm_layers'])
        dense_units = np.random.choice(param_dist['dense_units'])
        dropout_rate = np.random.choice(param_dist['dropout_rate'])

        model = create_lstm_model(hidden_units, lstm_layers, dense_units, dropout_rate)

        history_lstm = model.fit(X_train, y_train, epochs=num_epochs, batch_size=32, verbose=0)
        y_pred = model.predict(X_eval)
        y_pred_binary = (y_pred > 0.5).astype(int)
        accuracy = accuracy_score(y_eval, y_pred_binary)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model

    if best_model is not None:
        lstm_best_models.append(best_model)
        best_model.save(f"lstm_{season}")

    y_pred = best_model.predict(X_eval)
    y_pred_binary = (y_pred > 0.5).astype(int)

    accuracy = accuracy_score(y_eval, y_pred_binary)
    precision = precision_score(y_eval, y_pred_binary, average='macro')
    recall = recall_score(y_eval, y_pred_binary, average='macro')
    f1 = f1_score(y_eval, y_pred_binary, average='macro')

    conf_matrix = confusion_matrix(y_eval, y_pred_binary)

    all_metrics_lstm["Season"].append(season)
    all_metrics_lstm["Accuracy"].append(accuracy)
    all_metrics_lstm["Precision"].append(precision)
    all_metrics_lstm["Recall"].append(recall)
    all_metrics_lstm["F1-Score"].append(f1)

    print(f"Season {season} - Accuracy: {accuracy}")
    print(f"Season {season} - Recall: {recall}")
    print(f"Season {season} - F1-Score: {f1}")
    print(f"Season {season} - Confusion Matrix:\n{conf_matrix}")

In [ ]:
save_model_ann(lstm_best_models, "model_lstm")

In [ ]:
best_models_lstm = load_model_ann("model_lstm", SEASONS)

In [ ]:
# Save all_metrics_lstm
save_metrics(all_metrics_lstm, "all_metrics_lstm.pkl")

# Load all_metrics_lstm
loaded_all_metrics_lstm = load_metrics("all_metrics_lstm.pkl")

In [ ]:
import matplotlib.pyplot as plt

metrics = ["Accuracy", "Precision", "Recall", "F1-Score"]
colors = ['blue', 'green', 'orange', 'red']
markers = ['o', 's', '^', 'x']

plt.figure(figsize=(12, 8))

for i, metric in enumerate(metrics):
    plt.subplot(2, 2, i + 1)
    plt.bar(all_metrics_lstm["Season"], all_metrics_lstm[metric], color=colors[i], label=metric)
    plt.title(f'{metric} for Different Seasons')
    plt.xlabel('Seasons')
    plt.ylabel(metric)
    plt.ylim(0, 1) 
    plt.legend(loc='best')

plt.tight_layout()
plt.show()

- knn randomized search

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd
import numpy as np

SEASONS = ["Spring", "Summer", "Fall"]

param_dist = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2],
}

all_metrics_knn = {
    "Season": [],
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1-Score": [],
}

best_knn_models=[]

for season, train_data, eval_data in zip(SEASONS, train_df, eval_df):
    X_train = train_data.drop(columns="labels")
    y_train = train_data["labels"]
    X_eval = eval_data.drop(columns="labels")
    y_eval = eval_data["labels"]

    knn = KNeighborsClassifier()

    random_search = RandomizedSearchCV(knn, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', random_state=42, verbose=1)
    random_search.fit(X_train, y_train)

    best_knn = random_search.best_estimator_
    best_knn_models.append(best_knn)

    y_pred = best_knn.predict(X_eval)

    accuracy = accuracy_score(y_eval, y_pred)
    precision = precision_score(y_eval, y_pred, average='macro')
    recall = recall_score(y_eval, y_pred, average='macro')
    f1 = f1_score(y_eval, y_pred, average='macro')

    conf_matrix = confusion_matrix(y_eval, y_pred)

    all_metrics_knn["Season"].append(season)
    all_metrics_knn["Accuracy"].append(accuracy)
    all_metrics_knn["Precision"].append(precision)
    all_metrics_knn["Recall"].append(recall)
    all_metrics_knn["F1-Score"].append(f1)

    print(f"Season {season} - Accuracy: {accuracy}")
    print(f"Season {season} - Recall: {recall}")
    print(f"Season {season} - F1-Score: {f1}")
    print(f"Season {season} - Confusion Matrix:\n{conf_matrix}")


In [ ]:
save_models_joblib(best_knn_models,"knn")

In [ ]:
if not best_knn_models:
    best_knn_models = load_models_joblib("knn")

## attention

In [ ]:
!pip install transformers
!pip install pytorch-tabnet

### tabformer random search

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, recall_score, f1_score, confusion_matrix

best_models_tabnet = []

SEASONS = ["Spring", "Summer", "Fall"]

param_dist = {
    'n_d': [8, 16, 32, 64],
    'n_a': [8, 16, 32, 64],
    'n_steps': [3, 4, 5],
    'gamma': [1.0, 1.3, 1.5],
    'n_independent': [1, 2, 3],
}

scoring = {'accuracy': make_scorer(accuracy_score)}
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for i, (train_data, eval_data) in enumerate(zip(train_df, eval_df)):
    X_train = train_data.drop(columns="labels")
    y_train = train_data["labels"]
    X_eval = eval_data.drop(columns="labels")
    y_eval = eval_data["labels"]

    clf = TabNetClassifier()

    random_search = RandomizedSearchCV(
        clf,
        param_distributions=param_dist,
        scoring=scoring,
        refit='accuracy',
        n_iter=7, 
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        verbose=3,
        n_jobs=-1,
        random_state=42
    )

    random_search.fit(X_train.to_numpy(), y_train.to_numpy(),
                eval_set=[(X_eval.to_numpy(), y_eval.to_numpy())],eval_metric=['accuracy'],patience=20)


    best_tabnet = random_search.best_estimator_

    y_pred = best_tabnet.predict(X_eval.to_numpy())

    accuracy = accuracy_score(y_eval, y_pred)
    recall = recall_score(y_eval, y_pred, average='macro')
    f1 = f1_score(y_eval, y_pred, average='macro')
    conf_matrix = confusion_matrix(y_eval, y_pred)

    print(f"Model {i + 1} - Best Parameters: {random_search.best_params_}")
    print(f"Model {i + 1} - Accuracy: {accuracy}")
    print(f"Model {i + 1} - Recall: {recall}")
    best_models_tabnet.append(best_tabnet)


In [ ]:
save_models_joblib(best_models_tabnet, "tabnet")

In [ ]:
loaded_models_tabnet = load_models_joblib("tabnet")
loaded_models_tabnet

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, TFBertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import DataLoader, TensorDataset


In [ ]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize and prepare data
def tokenize_data(texts, labels, tokenizer, max_length=128):
    input_ids = []
    attention_masks = []
    labels = torch.tensor(labels)
    
    for text in texts:
        encoding = tokenizer(text, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])
    
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    dataset = TensorDataset(input_ids, attention_masks, labels)
    
    return dataset

train_dataset = tokenize_data(X_train, y_train, tokenizer)
eval_dataset = tokenize_data(X_eval, y_eval, tokenizer)

# Define DataLoader
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size)

# Set up training parameters
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
y_pred = []
y_true = []

for batch in eval_dataloader:
    input_ids, attention_mask, labels = batch
    input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
    
    y_pred.extend(logits.argmax(dim=1).tolist())
    y_true.extend(labels.tolist())

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
conf_matrix = confusion_matrix(y_true, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)


# Prediction

## Prepraing Test Data

In [ ]:
test_dataset_input_list = []
test_dataset_label_list = []
for season_name in NAME_SEASON:
    dataset = get_dataset(f'{TEST_OUTPUT_PATH}{season_name}*')
    # dataset.map(lambda input,label: tf.expand_dims(input, axis=0),label)
    # test_dataset_input_list.append(dataset)    
    dataset_input = dataset.map(lambda input,_: tf.expand_dims(input, axis=0))
    # pprint(dataset_input)
    dataset_label = dataset.map(lambda _,label: label)
    test_dataset_input_list.append(dataset_input)    
    test_dataset_label_list.append(dataset_label)

In [ ]:
test_dataset_spring = get_dataset(f'{TEST_OUTPUT_PATH}Spring*')
test_dataset_summer = get_dataset(f'{TEST_OUTPUT_PATH}Summer*')
test_dataset_fall = get_dataset(f'{TEST_OUTPUT_PATH}Fall*')
# test_balance_dataset(test_dataset_spring)
# test_balance_dataset(test_dataset_summer)
# test_balance_dataset(test_dataset_fall)

In [ ]:
def batch_dataset(dataset: tf.data.Dataset, batch_size) -> tuple[tf.data.Dataset, tf.data.Dataset]:
    indexed_dataset = dataset.enumerate()  # add an index to each example
    test_dataset = (
        indexed_dataset.filter(lambda i, _: i % 100 <= 100)
        .map(lambda _, data: data)  
        .batch(batch_size) 
    )
    return (test_dataset)

In [ ]:
test_dataset_all_seasons = [test_dataset_spring, test_dataset_summer, test_dataset_fall]
test_dataset_list = []

for dataset in test_dataset_all_seasons:
    test_season = batch_dataset(dataset, batch_size=32)
    test_dataset_list.append(test_season)

In [ ]:
test_dataset_list[0]

In [ ]:
test_df = [convert_dataset_to_dataframe(dataset) for dataset in test_dataset_list]

In [ ]:
test_df[1]['labels']

In [ ]:
def get_model_from_path(model_name):
    model_list = []
    for season_name in NAME_SEASON:
        model = tf.keras.models.load_model(f"{MODEL_SAVE_PATH}{model_name}/{season_name}")
        model_list.append(model)
    return model_list

In [ ]:
def generate_predict_list(model_list, test_dataset_input_list):
    predict_label_list = []
    for season, model, inputs in zip(NAME_SEASON, model_list, test_dataset_input_list):
        predict_label = model.predict(inputs)
        predict_label_list.append(predict_label)
    return predict_label_list

In [ ]:
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
import tensorflow.keras.metrics as metrics

def plot_confusion_matrix(test_dataset_label_list, predict_label_list_ann):
    """
    Plot confusion matrices for each pair of real and predicted labels in the given datasets.

    Args:
    test_dataset_label_list (list of tf.Tensor): A list of TensorFlow tensors containing real labels.
    predict_label_list_ann (list of tf.Tensor): A list of TensorFlow tensors containing predicted labels.

    """
    metric_list = [metrics.Recall(name = 'Recall'), metrics.BinaryAccuracy(name = 'Accuracy')]
    for real_labels, predict_labels in zip(test_dataset_label_list, predict_label_list_ann):
        predicted = np.array([1 if x >= 0.5 else 0 for x in predict_labels])
        actual = np.array([x for x in real_labels])
        
        for m in metric_list:
            m.update_state(actual, predicted)
            print(f'{m.name}: {m.result()}')
        
        conf_mat = confusion_matrix(actual, predicted)
        displ = ConfusionMatrixDisplay(confusion_matrix=conf_mat)
        displ.plot()

## Mia - ANN

### ANN baseline model

In [ ]:
model_ann_baseline_list = get_model_from_path('model_ann_baseline')

In [ ]:
predict_label_list_ann_baseline = generate_predict_list(model_ann_baseline_list, test_dataset_input_list)

In [ ]:
plot_confusion_matrix(test_dataset_label_list, predict_label_list_ann_baseline)

### ANN deeper model

In [ ]:
model_ann_deep_list = get_model_from_path('model_ann_deep')

In [ ]:
predict_label_list_ann_deep = generate_predict_list(model_ann_deep_list, test_dataset_input_list)

In [ ]:
plot_confusion_matrix(test_dataset_label_list, predict_label_list_ann_deep)

### Feature importance analysis

In [ ]:
np_test_input = np.asarray(list(test_dataset_input_list[0]))
np_test_input.shape

In [ ]:
def analyze_feature_importance(model, test_input, test_label, model_name, season_name, convert_back_to_tf_dataset: bool):
    results = []
    if convert_back_to_tf_dataset:
        pred = model.predict(tf.data.Dataset.from_tensor_slices(test_input), verbose=0).squeeze()
    else:
        pred = model.predict(test_input, verbose=0).squeeze()
        
    baseline_mae = np.mean(np.abs(pred-test_label ))
    results.append({'feature':'BASELINE','mae':baseline_mae})      
    
    for i, band in enumerate(BANDS):
        tmp_test_input = test_input[:,:,i].copy()
        np.random.shuffle(test_input[:,:,i])
        if convert_back_to_tf_dataset:
            pred = model.predict(tf.data.Dataset.from_tensor_slices(test_input), verbose=0).squeeze()
        else:
            pred = model.predict(test_input, verbose=0).squeeze()
                
        mae = np.mean(np.abs(pred - test_label))
        results.append({'feature':band,'mae':mae})
        print(f'{band}: {mae}')
        test_input[:,:,i] = tmp_test_input
    
    df = pd.DataFrame(results)
    df = df.sort_values('mae')
    plt.figure(figsize=(6,6))
    plt.barh(np.arange(len(BANDS)+1),df.mae)
    plt.yticks(np.arange(len(BANDS)+1),df.feature.values)
    plt.title(f'{model_name} {season_name} Feature Importance',size=16)
    plt.ylim((-1,len(BANDS)+1))
    plt.plot([baseline_mae,baseline_mae],[-1,len(BANDS)+1], '--', color='orange',
             label=f'Baseline OOF\nMAE={baseline_mae:.3f}')
    plt.xlabel(f'OOF MAE with feature permuted',size=14)
    plt.ylabel('Feature',size=14)
    plt.legend()
    plt.show()

In [ ]:
def analyze_ann_feature_importance(model_list, model_name, convert_back_to_tf_dataset: bool):
    for i, model in enumerate(model_list):
        analyze_feature_importance(model, 
                                   np.asarray(list(test_dataset_input_list[i])), 
                                   np.asarray(list(test_dataset_label_list[i])), model_name, NAME_SEASON[i], convert_back_to_tf_dataset)  

In [ ]:
analyze_ann_feature_importance(model_ann_baseline_list, 'ANN Baseline', False)

In [ ]:
analyze_ann_feature_importance(model_ann_deep_list, 'ANN Deep', True)

## Huizi - RF

In [ ]:
predicted_labels_per_season = {}

for season, data in zip(SEASONS, test_df):
    X_test = data.drop(columns="labels")
    y_test = data["labels"]

    rf = RandomForestClassifier(**best_params_per_season[season])
    rf.fit(X_train, y_train)  # Assuming the training data remains the same
    
    y_pred = rf.predict(X_test)
    predicted_labels_per_season[season] = y_pred

In [ ]:
all_metrics_test = {
    "Season": [],
    "Accuracy": [],
    "Recall": []
}

for season, data in zip(SEASONS, test_df):
    y_test = data["labels"]
    y_pred = predicted_labels_per_season[season]

    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    all_metrics_test["Season"].append(season)
    all_metrics_test["Accuracy"].append(accuracy)
    all_metrics_test["Recall"].append(recall)

metrics_test_table = pd.DataFrame(all_metrics_test)
print(metrics_test_table)

In [ ]:
for season, data in zip(SEASONS, test_df):
    y_test = data["labels"]
    y_pred = predicted_labels_per_season[season]

    conf_mat = confusion_matrix(y_test, y_pred)
    displ = ConfusionMatrixDisplay(confusion_matrix=conf_mat)
    
    plt.figure()
    displ.plot()
    plt.title(f'Confusion Matrix for {season}')

In [ ]:
# Predict using the default parameters of RandomForest
predicted_labels_default_per_season = {}

for season, data in zip(SEASONS, test_df):
    X_test = data.drop(columns="labels")
    y_test = data["labels"]

    # Using default RandomForestClassifier
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)  # Assuming the training data remains the same
    
    y_pred = rf.predict(X_test)
    predicted_labels_default_per_season[season] = y_pred

# Compute metrics for predictions with default parameters
all_metrics_test_default = {
    "Season": [],
    "Accuracy": [],
    "Recall": []
}

for season, data in zip(SEASONS, test_df):
    y_test = data["labels"]
    y_pred = predicted_labels_default_per_season[season]

    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    all_metrics_test_default["Season"].append(season)
    all_metrics_test_default["Accuracy"].append(accuracy)
    all_metrics_test_default["Recall"].append(recall)

# Convert to DataFrame and display
metrics_test_default_table = pd.DataFrame(all_metrics_test_default)
print(metrics_test_default_table)

# Display confusion matrices for predictions with default parameters
for season, data in zip(SEASONS, test_df):
    y_test = data["labels"]
    y_pred = predicted_labels_default_per_season[season]

    conf_mat = confusion_matrix(y_test, y_pred)
    displ = ConfusionMatrixDisplay(confusion_matrix=conf_mat)
    
    plt.figure()
    displ.plot()
    plt.title(f'Confusion Matrix (Default Params) for {season}')
    plt.show()


## Justin

ANN2

In [ ]:
model_ann2_list = []
for season_name in NAME_SEASON:
    model = tf.keras.models.load_model(f"{MODEL_SAVE_PATH}model_ann2/{season_name}")
    pprint(model)
    model_ann2_list.append(model)

In [ ]:
predict_label_list_ann2 = generate_predict_list(model_ann2_list, test_dataset_input_list)

In [ ]:
plot_confusion_matrix(test_dataset_label_list, predict_label_list_ann2)

XGBOOST

In [ ]:
best_models_xgb

In [ ]:
test_season_feature_importances_xgb = {}

for i, (test_dataframe_season, model) in enumerate(zip(test_df, best_models_xgb)):
    test_dataframe_season = pd.get_dummies(test_dataframe_season, columns=['labels'])
    
    test_dataframe_X = test_dataframe_season.loc[:, ~test_dataframe_season.columns.str.startswith('labels_')]
    test_dataframe_Y = test_dataframe_season.loc[:, test_dataframe_season.columns.str.startswith('labels_')]
    
    test_prediction = model.predict(xgb.DMatrix(test_dataframe_X), output_margin=True)
    # Use output_margin=True to obtain probability scores instead of class labels
    
    accuracy = accuracy_score(test_dataframe_Y, (test_prediction > 0.5).astype(int))
    recall = recall_score(test_dataframe_Y, (test_prediction > 0.5).astype(int), average='macro')
    
    conf_matrix = confusion_matrix(test_dataframe_Y.values.argmax(axis=1), (test_prediction > 0.5).astype(int).argmax(axis=1))
    # feature_importance = model.feature_importances_
    
    print(f"Model {i + 1} - Accuracy: {accuracy}")
    print(f"Model {i + 1} - Recall: {recall}")
    print(f"Model {i + 1} - Confusion Matrix:\n{conf_matrix}")
    
    # test_season_feature_importances_xgb[f'Model {i + 1}'] = feature_importance

knn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
import pandas as pd
from pprint import pprint

predicted_labels_per_season = {}

# Define data for test seasons
test_seasons = test_df  # Assuming you have test data for each season in the test_df list

all_metrics_test = {
    "Season": [],
    "Accuracy": [],
    "Recall": [],
    "Confusion Matrix": []
}

for season, data in zip(SEASONS, test_seasons):
    X_test = data.drop(columns="labels").to_numpy()
    y_test = data["labels"].to_numpy()

    knn_model = best_knn_models[SEASONS.index(season)]

    y_pred = knn_model.predict(X_test)
    predicted_labels_per_season[season] = y_pred

for season, data in zip(SEASONS, test_seasons):
    y_test = data["labels"].to_numpy()
    y_pred = predicted_labels_per_season[season]

    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    confusion_matrix_result = confusion_matrix(y_test, y_pred)

    all_metrics_test["Season"].append(season)
    all_metrics_test["Accuracy"].append(accuracy)
    all_metrics_test["Recall"].append(recall)
    all_metrics_test["Confusion Matrix"].append(confusion_matrix_result)

metrics_test_table = pd.DataFrame(all_metrics_test)
pprint(metrics_test_table)

tabnet

In [ ]:
if not best_models_tabnet:
    best_models_tabnet = load_models_joblib("tabnet")

In [ ]:
from sklearn.metrics import confusion_matrix

predicted_labels_per_season = {}

# Define data for test seasons
test_seasons = test_df  # Assuming you have test data for each season in the test_df list

all_metrics_test = {
    "Season": [],
    "Accuracy": [],
    "Recall": [],
    "Confusion Matrix": []
}

for season, data in zip(SEASONS, test_seasons):
    X_test = data.drop(columns="labels").to_numpy()
    y_test = data["labels"].to_numpy()

    # Retrieve the best TabNet model for the current season
    tabnet_model = best_models_tabnet[SEASONS.index(season)]

    y_pred = tabnet_model.predict(X_test)
    predicted_labels_per_season[season] = y_pred

for season, data in zip(SEASONS, test_seasons):
    y_test = data["labels"].to_numpy()
    y_pred = predicted_labels_per_season[season]

    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    confusion_matrix_result = confusion_matrix(y_test, y_pred)

    all_metrics_test["Season"].append(season)
    all_metrics_test["Accuracy"].append(accuracy)
    all_metrics_test["Recall"].append(recall)
    all_metrics_test["Confusion Matrix"].append(confusion_matrix_result)

metrics_test_table = pd.DataFrame(all_metrics_test)
pprint(metrics_test_table)

# Mia - Fine tuning Deeper ANN

In [ ]:
import keras_tuner 


def call_existing_code(units_0, layer_num, activation, dropout, dropout_rate, lr):
    model = keras.Sequential()
    model.add(layers.Dense(units=units_0, input_dim=NUM_INPUTS, activation='relu', name='Dense0'))
    cur_layer = 1
    while(cur_layer < layer_num):
        model.add(layers.Dense(
            units=hp.Int(f"units_{cur_layer}", min_value=32, max_value=512, step=32), 
            activation='relu', 
            name=f'Dense{cur_layer}'))
        if dropout:
            model.add(layers.Dropout(rate=dropout_rate))
        model.add(layers.BatchNormalization())
        cur_layer += 1
    model.add(layers.Dense(1, activation = 'sigmoid', name='Output'))
    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate=lr), 
        metrics=[
            tf.keras.metrics.BinaryAccuracy(name=all_metric_names[0]), 
            tf.keras.metrics.Recall(name=all_metric_names[1]),
            tf.keras.metrics.AUC(name=all_metric_names[2]),
            tf.keras.metrics.FalseNegatives(name=all_metric_names[3])], 
        loss ='binary_crossentropy',
    )
    return model


def build_model(hp):
    layer_num = hp.Int("layer_num", min_value=1, max_value=6, step=1)
    units_0 = hp.Int("units_0", min_value=32, max_value=512, step=32)
    activation = hp.Choice("activation", ["relu", "elu"])
    dropout = hp.Boolean("dropout")
    dropout_rate = hp.Float("dropout_rate", min_value=0.05, max_value=0.25, step=0.05)
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    
    model = call_existing_code(
        units_0=units_0, layer_num=layer_num, activation=activation, dropout=dropout, dropout_rate=dropout_rate, lr=lr
    )
    return model

In [ ]:
hp = keras_tuner.HyperParameters()

In [ ]:
def get_random_search_instance(objective, season_name, overwrite=False):
    metric_name = ''
    if isinstance(objective, str):
        metric_name = objective
    else:
        for i, objective_tune in enumerate(objective):
            if i != 0:
                metric_name += '_'
            metric_name += objective_tune.name
    return keras_tuner.RandomSearch(
        hypermodel=build_model,
        objective=objective,
        max_trials=10,
        overwrite=overwrite,
        directory="./model/finetune",
        project_name=f"finetune_{metric_name}_{season_name}",
    )

In [ ]:
def start_fine_tune(objective, epochs=10):
    for i, (train_dataset, eval_dataset) in enumerate(zip(train_dataset_list, eval_dataset_list)):
        tf.keras.backend.clear_session()
        tuner = get_random_search_instance(objective, NAME_SEASON[i], True)
        tuner.search(train_dataset, validation_data=eval_dataset, epochs=epochs)

In [ ]:
def load_best_model(objective):
    model_ann_finetuned_list = []
    for i, (train_dataset, eval_dataset) in enumerate(zip(train_dataset_list, eval_dataset_list)):
        tf.keras.backend.clear_session()
        tuner = get_random_search_instance(objective, NAME_SEASON[i], False)
        models = tuner.get_best_models()
        best_model = models[0]
        best_model.build()
        best_model.summary()
        model_ann_finetuned_list.append(best_model)
    return model_ann_finetuned_list

In [ ]:
def predict_with_best_model(model_finetuned_list):
    predict_label_list = generate_predict_list(model_finetuned_list, test_dataset_input_list)
    plot_confusion_matrix(test_dataset_label_list, predict_label_list)

## Fine tune with objective as 'val_Recall'

In [ ]:
finetune_objective = "val_Recall"

### Fine tune

In [ ]:
start_fine_tune(finetune_objective)

### Load best results and predict

In [ ]:
finetuned_recall_model_list = load_best_model(finetune_objective)
predict_with_best_model(finetuned_recall_model_list)

## Fine tune with objective as 'val_Accuracy'

In [ ]:
finetune_objective = "val_Accuracy"

### Fine tune

In [ ]:
start_fine_tune(finetune_objective)

### Load best results and predict

In [ ]:
finetuned_recall_model_list = load_best_model(finetune_objective)
predict_with_best_model(finetuned_recall_model_list)

## Fine tune with both 'val_Accuracy', 'val_Recall' and 'val_AUC'

In [ ]:
finetune_objective = [
    keras_tuner.Objective('val_Accuracy', "max"), 
    keras_tuner.Objective('val_Recall', "max"), 
    keras_tuner.Objective('val_AUC', "max")]

### Fine tune

In [ ]:
start_fine_tune(finetune_objective)

### Load best results and predict

In [ ]:
finetuned_recall_model_list = load_best_model(finetune_objective)
predict_with_best_model(finetuned_recall_model_list)

# Mia - FTTransformer

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from fttransformer.utils.preprocessing import df_to_dataset, build_categorical_prep
from fttransformer.models.fttransformer import FTTransformerEncoder, FTTransformer

In [ ]:
NUM_INPUTS = len(feature_list)
LABEL = fire_class_band
BANDS = []
for feature in feature_list:
  BANDS += [feature['band']]
FEATURE_NAMES = BANDS + [LABEL]

NUMERIC_FEATURES = []
CATEGORICAL_FEATURES = []

def separate_feature():
    for band in BANDS:
        if band == 'LC_Type1':
            CATEGORICAL_FEATURES.append(band)
        else:
            NUMERIC_FEATURES.append(band)
separate_feature()
print(NUMERIC_FEATURES)

columns = [tf.io.FixedLenFeature(shape=[], dtype=tf.float32) for k in FEATURE_NAMES]
FEATURES_DICT = dict(zip(FEATURE_NAMES, columns))
print(FEATURES_DICT)

BATCH_SIZE = 32

CSV_EXPORT_FOLDER = 'processed_datasets/'

## Parse tfrecord and export preprocessed data

In [ ]:
def parse_tfrecord(example_proto):
    """The parsing function.
    Read a serialized example into the structure defined by FEATURES_DICT.
    Args:
    example_proto: a serialized Example.
    Returns:
    A dictionary of tensors, keyed by feature name.
    """
    parsed_features = tf.io.parse_single_example(example_proto, FEATURES_DICT)
    return parsed_features

def get_dataset_df(pattern):
    """Function to read, parse and format to tuple a set of input tfrecord files.
    Get all the files matching the pattern, parse and convert to tuple.
    Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
    Returns:
    A tf.data.Dataset
    """
    print(pattern)
    glob = tf.io.gfile.glob(pattern)
    dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
    dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
    return tfds.as_dataframe(dataset)

In [ ]:
train_dataset_spring = get_dataset_df(f'{TRAINING_OUTPUT_PATH}Spring*')
train_dataset_summer = get_dataset_df(f'{TRAINING_OUTPUT_PATH}Summer*')
train_dataset_fall = get_dataset_df(f'{TRAINING_OUTPUT_PATH}Fall*')
print(train_dataset_spring.head())
train_dataset_all_seasons = [train_dataset_spring, train_dataset_summer, train_dataset_fall]

In [ ]:
from sklearn.preprocessing import (MinMaxScaler, RobustScaler)

def get_normalize_features(numerical_feature_list, fit_target, transform_list):
    norm = RobustScaler().fit(fit_target[numerical_feature_list])
    norm_list=[]
    for transform_target in transform_list:
        transformed = transform_target.copy()
        transformed[numerical_feature_list] = norm.transform(transform_target[numerical_feature_list])
        norm_list.append(transformed)
    
    return norm_list

In [ ]:
for season, train_set, test_set in zip(NAME_SEASON, train_dataset_all_seasons, test_dataset_all_seasons):
    [train_norm, test_norm] = get_normalize_features(NUMERIC_FEATURES, train_set, [train_set, test_set])
    print(train_norm.head())
    print(test_norm.head())
    test_norm.to_csv(f'{CSV_EXPORT_FOLDER}{season}_test_set.csv', index=False)    
    train_norm.to_csv(f'{CSV_EXPORT_FOLDER}{season}_train_set.csv', index=False) 

## Read normalized dataset from disk

In [ ]:
from sklearn.model_selection import train_test_split
train_all_season = []
val_all_season = []
test_all_season = []

for season in NAME_SEASON:
    df = pd.read_csv(f'{CSV_EXPORT_FOLDER}{season}_train_set.csv')
    df[CATEGORICAL_FEATURES] = df[CATEGORICAL_FEATURES].astype(str)
    data_tr, data_val = train_test_split(df, test_size=0.2)
    train_all_season.append(data_tr)
    val_all_season.append(data_val)
    df = pd.read_csv(f'{CSV_EXPORT_FOLDER}{season}_test_set.csv')
    df[CATEGORICAL_FEATURES] = df[CATEGORICAL_FEATURES].astype(str)
    test_all_season.append(df)
    print(f'{season}: \
          {len(train_all_season[len(train_all_season)-1])},\
          {len(val_all_season[len(val_all_season)-1])}, \
          {len(test_all_season[len(test_all_season)-1])}')

In [ ]:
def get_batched_data(df_tr, df_val, batch_size=256):
    tfds_train = df_to_dataset(df_tr[FEATURE_NAMES], LABEL, shuffle=True, batch_size=batch_size)
    tfds_val = df_to_dataset(df_val[FEATURE_NAMES], LABEL, shuffle=False, batch_size=batch_size)
    return tfds_train, tfds_val

In [ ]:
from fttransformer.models.fttransformer import FTTransformerEncoder, FTTransformer

def call_existing_code(
    df_train,
    d_embedding:int, 
    n_layers:int, 
    ffn_factor:float, 
    attention_dropout:float, 
    ffn_dropout:float, 
    residual_dropout:float, 
    weight_decay:float, 
    lr:float):
    
    ft_linear_encoder = FTTransformerEncoder(
        numerical_features=NUMERIC_FEATURES,  # list of numeric features
        categorical_features=CATEGORICAL_FEATURES,  # list of numeric features
        numerical_data=df_train[NUMERIC_FEATURES].values,
        categorical_data=df_train[CATEGORICAL_FEATURES].values,
        y = None,
        numerical_embedding_type='linear',
        embedding_dim=d_embedding,
        depth=n_layers,
        ffn_factor=ffn_factor,
        attn_dropout=attention_dropout,
        ff_dropout=ffn_dropout,
        residual_dropout=residual_dropout,
        explainable=True
    )

    ft_model = FTTransformer(
        encoder=ft_linear_encoder,  # Encoder from above
        out_dim=1,  # Number of outputs in final layer
        out_activation='sigmoid',  # Activation function for final layer
    )
    
    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=lr, weight_decay=weight_decay
    )

    ft_model.compile(
        optimizer = optimizer,
        # metrics=[
        #     # tf.keras.metrics.BinaryAccuracy(name=all_metric_names[0]), 
        #     tf.keras.metrics.Recall(name=all_metric_names[1]),
        #     tf.keras.metrics.AUC(name=all_metric_names[2]),
        #     # tf.keras.metrics.FalseNegatives(name=all_metric_names[3])
        # ], 
        metrics= {"output": [
            tf.keras.metrics.BinaryAccuracy(name=all_metric_names[0]),
            tf.keras.metrics.Recall(name=all_metric_names[1]),
            tf.keras.metrics.AUC(name=all_metric_names[2], curve='PR'),
            tf.keras.metrics.FalseNegatives(name=all_metric_names[3])
        ], "importances": None},
        loss = {"output": tf.keras.losses.BinaryCrossentropy(), "importances": None},
    )
    
    return ft_model

In [ ]:
def call_default_ft(df_train, data_train, data_val):
    lr = 0.001
    weight_decay = 0.0001
    ft_model = call_existing_code(
        df_train,
        d_embedding=32, 
        n_layers=4, 
        ffn_factor=1,
        attention_dropout=0.1,
        ffn_dropout=0.1,
        residual_dropout=0.0,
        weight_decay=weight_decay,
        lr=lr
    )
    epochs = 200
    
    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor="val_output_Accuracy", mode="max", patience=20, restore_best_weights=True),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=f'{MODEL_SAVE_PATH}{MODEL_NAME}',
            save_weights_only=True,
            save_freq='epoch')
    ]
    history = ft_model.fit(
        data_train, 
        epochs=epochs, 
        validation_data=data_val,
        callbacks=[callbacks]
    )
    return ft_model, history

In [ ]:
all_metric_names = ['Accuracy', 'Recall', 'AUC', 'FalseNegatives']
prefix = 'output_'
def plot_metrics(history, metric_name, plot_title):
    if f'{prefix}{metric_name}' not in history.history:
        print()
        return
    plt.figure(figsize=(6, 4))
    plt.plot(history.history[f'{prefix}{metric_name}'], label=f'training {metric_name}')
    plt.plot(history.history[f'val_{prefix}{metric_name}'], label=f'validation {metric_name}')
    plt.xlabel('Epoch')
    plt.ylabel(f'{metric_name}')
    plt.title(f'Training and Validation {plot_title} {metric_name}')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_model_history(history, plot_title):
    for metric_name in all_metric_names:
        plot_metrics(history, metric_name, plot_title)

In [ ]:
MODEL_SAVE_PATH = 'model/'
MODEL_NAME = 'ftmodel_baseline'
ftmodel_baseline_list = []
for season, train_df, val_df in zip(NAME_SEASON, train_all_season, val_all_season):
    tf.keras.backend.clear_session()    
    print(train_df.shape)
    print(val_df.shape)
    tfds_train, tfds_val = get_batched_data(train_df, val_df)
    model, history = call_default_ft(train_df, tfds_train, tfds_val)
    ftmodel_baseline_list.append(model)
    plot_model_history(history, season)

In [ ]:
save_model(ftmodel_baseline_list, MODEL_NAME)

## Predict

In [ ]:
test_dataset_input_list = []
test_dataset_label_list = []
for dataset in test_all_season:
    dataset_tf = df_to_dataset(dataset[BANDS], shuffle=False, batch_size=1)
    test_dataset_input_list.append(dataset_tf)
    test_dataset_label_list.append(dataset[[LABEL]])

In [ ]:
MODEL_NAME = 'ftmodel_baseline'
model_fttransformer_baseline_list = get_model_from_path(MODEL_NAME)

In [ ]:
predict_ftbaseline_list = generate_predict_list(model_fttransformer_baseline_list, test_dataset_input_list)

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, recall_score
from fttransformer.utils.helper import get_model_importances

def evaluate_result(actual_label_list, predict_label_list):
    for season, actual, prediction in zip(NAME_SEASON, actual_label_list, predict_label_list):
        print(season)
        print("Test Accuracy:", np.round(accuracy_score(actual, prediction['output'].ravel()>0.5), 4))
        print("Test Recall:", np.round(recall_score(actual, prediction['output'].ravel()>0.5), 4))
        print("Test ROC AUC:", np.round(roc_auc_score(actual, prediction['output'].ravel()), 4))
        print("Test PR AUC:", np.round(average_precision_score(actual, prediction['output'].ravel()), 4))
        
def plot_feature_importance(actual_label_list, predict_label_list):
    for season, actual, prediction in zip(NAME_SEASON, actual_label_list, predict_label_list):
        linear_importances = prediction['importances']
        linear_importances_df = pd.DataFrame(linear_importances[:, :-1], columns = BANDS)
        linear_total_importances = get_model_importances(
            linear_importances_df, title=f"{season} Importances for FT-Transformer with Linear Numerical Embedddings"
        )
        print(f"{season}")        
        print(linear_total_importances)

In [ ]:
evaluate_result(test_dataset_label_list, predict_ftbaseline_list)
plot_feature_importance(test_dataset_label_list, predict_ftbaseline_list)

## Fine tune FTTransformer

In [ ]:
import keras_tuner 

class MyHyperModel(keras_tuner.HyperModel):
    def __init__(self, df_train):
        self.df_train = df_train
        
    def build(self, hp):
        d_embedding = hp.Int("d_embedding", min_value=32, max_value=512, step=8)
        n_layers = hp.Int("n_layers", min_value=1, max_value=4)
        ffn_factor = hp.Float("ffn_factor", min_value=2/3, max_value=8/3, sampling="linear")
        attention_dropout = hp.Float("attention_dropout", min_value=0.0, max_value=0.5, sampling="linear")
        ffn_dropout = hp.Float("ffn_dropout", min_value=0.0, max_value=0.5, sampling="linear")
        residual_dropout = hp.Float("residual_dropout", min_value=0.0, max_value=0.2, default=0.0, sampling="linear")

        lr = hp.Float("lr", min_value=1e-5, max_value=1e-2, sampling="log")
        weight_decay = hp.Float("weight_decay", min_value=1e-6, max_value=1e-3, sampling="log")

        model = call_existing_code(
            self.df_train,
            d_embedding=d_embedding, 
            n_layers=n_layers, 
            ffn_factor=ffn_factor,
            attention_dropout=attention_dropout,
            ffn_dropout=ffn_dropout,
            residual_dropout=residual_dropout,
            weight_decay=weight_decay,
            lr=lr
        )
        return model

In [ ]:
def get_random_search_instance(objective, hp, df_train, season_name, overwrite=False):
    metric_name = ''
    if isinstance(objective, str):
        metric_name = objective
    else:
        for i, objective_tune in enumerate(objective):
            if i != 0:
                metric_name += '_'
            metric_name += objective_tune.name
    m = MyHyperModel(df_train)
    return keras_tuner.RandomSearch(
        hypermodel=m,
        objective=objective,
        max_trials=30,
        overwrite=overwrite,
        directory="./model/finetune-fttransformer",
        project_name=f"finetune_{metric_name}_{season_name}",
    )

In [ ]:
def start_fine_tune(objective, epochs=10):
    for season, train_df, val_df in zip(NAME_SEASON, train_all_season, val_all_season):
        tf.keras.backend.clear_session()    
        hp = keras_tuner.HyperParameters()        
        tfds_train, tfds_val = get_batched_data(train_df, val_df)
        tuner = get_random_search_instance(objective, hp, train_df, season, True)
        tuner.search(tfds_train, validation_data=tfds_val, epochs=epochs)

In [ ]:
def load_best_model(objective):
    best_models = []
    for season, train_df, val_df in zip(NAME_SEASON, train_all_season, val_all_season):
        tf.keras.backend.clear_session()
        tfds_train, tfds_val = get_batched_data(train_df, val_df)
        hp = keras_tuner.HyperParameters()        
        tuner = get_random_search_instance(objective, hp, train_df, season, False)
        models = tuner.get_best_models()
        best_model = models[0]
        input_tensors ={}
        for key,value in tfds_train.element_spec[0].items():
            input_tensors[key] = tf.keras.layers.Input(type_spec = value)
            shape = value.shape
        best_model.call(input_tensors)
        # best_model.build(shape)
        # best_model.summary()
        best_models.append(best_model)
    return best_models

In [ ]:
def resume_tuning(objective, epochs=10):
    for season, train_df, val_df in zip(NAME_SEASON, train_all_season, val_all_season):
        tf.keras.backend.clear_session()    
        tfds_train, tfds_val = get_batched_data(train_df, val_df)
        hp = keras_tuner.HyperParameters()        
        tuner = get_random_search_instance(objective, hp, train_df, season, False)
        tuner.search(tfds_train, validation_data=tfds_val, epochs=epochs)

In [ ]:
finetune_objective = [
    keras_tuner.Objective('val_output_Accuracy', "max"),
    keras_tuner.Objective('val_output_Recall', "max"),
]

In [ ]:
start_fine_tune(finetune_objective, epochs=20)

In [ ]:
resume_tuning(finetune_objective, epochs=20)

### Load best results and predict

In [ ]:
from fttransformer.models.fttransformer import FTTransformer
finetuned_recall_model_list = load_best_model(finetune_objective)

In [ ]:
predict_ft_tuned_list = generate_predict_list(finetuned_recall_model_list, test_dataset_input_list)

In [ ]:
evaluate_result(test_dataset_label_list, predict_ft_tuned_list)
plot_feature_importance(test_dataset_label_list, predict_ft_tuned_list)

In [ ]:
def plot_confusion_matrix(test_dataset_label_list, predict_ft_tuned_list):
    metric_list = [metrics.Recall(name = 'Recall'), metrics.BinaryAccuracy(name = 'Accuracy')]
    for real_labels, predict_labels in zip(test_dataset_label_list, predict_ft_tuned_list):
        real_values = real_labels['is_fire'].values
        predicted = np.array([1 if x>0.5 else 0 for x in predict_labels['output'].ravel()])
        actual = np.array([x for x in real_values])
        print(predicted)
        print(real_values)
        for m in metric_list:
            m.update_state(actual, predicted)
            print(f'{m.name}: {m.result()}')

        conf_mat = confusion_matrix(actual, predicted)
        displ = ConfusionMatrixDisplay(confusion_matrix=conf_mat)
        displ.plot()

plot_confusion_matrix(test_dataset_label_list, predict_ft_tuned_list)